# Просмотр сгенерированных рекомендаций

Для просмотра сгенерированных рекомендаций будет использоваться пробный запуск Kedro через Python.

In [1]:
%%capture
%load_ext kedro.ipython
%reload_kedro ..

In [2]:
%run_viz

[02/26/23 15:00:28] WARNING  /opt/homebrew/Caskroom/miniconda/ba ]8;id=117207;file:///opt/homebrew/Caskroom/miniconda/base/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=754282;file:///opt/homebrew/Caskroom/miniconda/base/lib/python3.8/warnings.py#109\109]8;;\
                             se/lib/python3.8/site-packages/jupy                
                             ter_client/connect.py:27:                          
                             migrating its paths to use standard                
                             platformdirs                                       
                             given by the platformdirs library.                 
                             To remove this warning and                         
                             see the appropriate new                            
                             directories, set the environment                   
                             variable                                           


In [35]:
%%capture

from kedro.io import DataCatalog, MemoryDataSet
from kedro.extras.datasets.pandas import CSVDataSet

from kedro.runner import SequentialRunner
from kedro.framework.project import pipelines


input_csv = "Anime/data/extended_anime_2.csv"
col = "Tags"


io = DataCatalog(
        {
            'dataframe': CSVDataSet(
                filepath=input_csv, load_args={'index_col': 'Rank'}),
            'dataframe_with_recs': MemoryDataSet(),
            'params:target_column': MemoryDataSet(col)
        }
    )

    
default_pipeline = pipelines['__default__']

SequentialRunner().run(default_pipeline, catalog=io)

In [2]:
df_with_recs = io.load('dataframe_with_recs')

NameError: name 'io' is not defined

In [32]:
df_with_recs.head(3)

,Name,Japanese_name,Type,Episodes,Studio,Release_season,Tags,Rating,Release_year,End_year,...,Related_anime,Voice_actors,staff,Russian_name,Russian_description,Image_url,Shikimori_url,Franchise,Shikimori_rating,recommendations
0,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Kimetsu no Yaiba: Yuukaku-hen,TV,NaN,ufotable,Fall,"Action, Adventure, Fantasy, Shounen, Demons, H...",4.60,2021.0,NaN,...,"Demon Slayer: Kimetsu no Yaiba, Demon Slayer: ...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk...","Koyoharu Gotouge : Original Creator, Haruo Sot...","Клинок, рассекающий демонов: Квартал красных ф...","<div class=""b-text_with_paragraphs"">Продолжени...",https://shikimori.one/system/animes/original/4...,https://shikimori.one/animes/47778-kimetsu-no-...,demon_slayer,8.82,"[13, 15011, 3062, 6269, 4422, 7592, 8325, 1013..."
1,Fruits Basket the Final Season,Fruits Basket the Final,TV,13.0,TMS Entertainment,Spring,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",4.60,2021.0,NaN,...,"Fruits Basket 1st Season, Fruits Basket 2nd Se...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum...","Natsuki Takaya : Original Creator, Yoshihide I...",Корзинка фруктов: Финал,"<div class=""b-text_with_paragraphs"">Сотни лет ...",https://shikimori.one/system/animes/original/4...,https://shikimori.one/animes/42938-fruits-bask...,fruits_basket,9.01,"[27, 108, 911, 11948, 5726, 2540, 14694, 4238,..."
2,Mo Dao Zu Shi 3,The Founder of Diabolism 3,Web,12.0,B.C MAY PICTURES,NaN,"Fantasy, Ancient China, Chinese Animation, Cul...",4.58,2021.0,NaN,...,"Mo Dao Zu Shi 2, Mo Dao Zu Shi Q","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang...","Mo Xiang Tong Xiu : Original Creator, Xiong Ke...",Магистр дьявольского культа 3,"<div class=""b-text_with_paragraphs"">Жизнь закл...",https://shikimori.one/system/animes/original/4...,https://shikimori.one/animes/40434-mo-dao-zu-s...,mo_dao_zu_shi,8.63,"[3205, 2181, 2235, 3722, 2105, 2379, 2678, 682..."


In [4]:
from typing import List

import pandas as pd

def get_recommendations(
        title: str, title_column_name: str,
        recs_column_name: str, df: pd.DataFrame) -> List[str]:
    
    matches = df[df[title_column_name] == title]
    if len(matches) == 0:
        matches = df[df[title_column_name].str.contains(title)]

    if len(matches) > 1:
        print(
            "Too much results ({}). You must choose the only one:\n{}".format(
                len(matches), "\n".join(matches[title_column_name].to_list())
            )
        )
        return
    
    item_series = matches.iloc[0]
    recs = item_series[recs_column_name]

    item_series_by_recs = df.loc[recs]

    return item_series_by_recs[title_column_name].to_list()


title = "Another"

get_recommendations(
    title,
    "Name",
    "recommendations",
    df_with_recs
)

['Shiki',
 'Higurashi: When They Cry - Sotsu',
 'Boogiepop and Others',
 'Owarimonogatari',
 'Shiki Specials',
 'Digital Devil Story Megami Tensei',
 'The Garden of Sinners Movie 1: Thanatos. (Overlooking View)',
 'Red Garden',
 'Higurashi: When They Cry - Gou',
 'When They Cry: Higurashi']